<a href="https://colab.research.google.com/github/HowardHNguyen/ML_DL_Time_Series_by_Python/blob/main/COMPARISON_SARIMA%2C_FB_Prophet%2C_and_NeuralProphet_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMPARISON: SARIMA, FB Prophet, and NeuralProphet models**

1. We loaded and prepared the dataset.
2. Ensured consistent date parsing.
3. Removed duplicate dates and aggregated data.
4. Fitted the SARIMA, Prophet, and NeuralProphet models.
5. Generated forecasts for each model.
6. Aligned the forecasts with the actual data length for accurate error metrics calculation.
7. Calculated and displayed the error metrics.
8. Plotted the forecast comparisons using Plotly.

Below is the complete implementation:

In [2]:
!pip install neuralprophet plotly pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 24.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objs as go
from datetime import datetime, timedelta
from prophet import Prophet
from neuralprophet import NeuralProphet

In [11]:
# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/data/flu-ili-byregion-fluseason_data.csv')

# Convert weekending to datetime format
df['weekending'] = pd.to_datetime(df['weekending'], format='%m/%d/%y')

# Filter data for California region
df_california = df[df['region'] == 'California'][['weekending', 'Total_ILI']].rename(columns={'weekending': 'ds', 'Total_ILI': 'y'})

# Ensure date parsing is consistent
df_california['ds'] = pd.to_datetime(df_california['ds'], format='%Y-%m-%d')

# Identify duplicate dates
duplicates = df_california[df_california.duplicated(subset=['ds'], keep=False)]
print(f"Duplicate dates found:\n{duplicates}")

# Remove duplicates by aggregating (e.g., taking the mean of duplicate entries)
df_california = df_california.groupby('ds', as_index=False).agg({'y': 'mean'})

# Fit SARIMA model
sarima_model = SARIMAX(df_california['y'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 52))
sarima_fit = sarima_model.fit()

# Create future dataframe for forecast up to end of 2025
last_date = pd.to_datetime(df_california['ds'].max())
end_date = datetime(2025, 12, 31)

# Calculate the number of periods (weeks) between last_date and end_date
n_periods = ((end_date - last_date).days // 7) + 1  # Adding 1 to include the end_date

# Generate future dates
future_dates = pd.date_range(start=last_date + timedelta(weeks=1), periods=n_periods, freq='W')

# Generate forecast for SARIMA
sarima_forecast = sarima_fit.get_forecast(steps=n_periods)
sarima_forecast_mean = sarima_forecast.predicted_mean
sarima_forecast_ci = sarima_forecast.conf_int()

# Convert SARIMA forecast to DataFrame
sarima_forecast_df = pd.DataFrame({'ds': future_dates, 'yhat': sarima_forecast_mean})

# Initialize and fit Prophet model
prophet_model = Prophet()
prophet_model.fit(df_california)

# Create future dataframe for forecast up to end of 2025
future_dates_prophet = prophet_model.make_future_dataframe(periods=n_periods, freq='W')

# Generate forecast for Prophet
prophet_forecast = prophet_model.predict(future_dates_prophet)
prophet_forecast_df = prophet_forecast[['ds', 'yhat']]

# Initialize and fit NeuralProphet model
neuralprophet_model = NeuralProphet()

# Ensure date format consistency in df_california
df_california['ds'] = pd.to_datetime(df_california['ds'], format='%Y-%m-%d')

neuralprophet_model.fit(df_california, freq='W')

# Create future dataframe for forecast up to end of 2025
future_dates_df = neuralprophet_model.make_future_dataframe(df_california, periods=n_periods)

# Generate forecast for NeuralProphet
neuralprophet_forecast = neuralprophet_model.predict(future_dates_df)
neuralprophet_forecast_df = neuralprophet_forecast[['ds', 'yhat1']]

# Define function to calculate error metrics
def calculate_metrics(actual, forecast):
    mae = mean_absolute_error(actual, forecast)
    mse = mean_squared_error(actual, forecast)
    rmse = np.sqrt(mse)
    return mae, mse, rmse

# Actual values for comparison
actual_values = df_california['y'].values

# Align forecast with actual data length for SARIMA
sarima_forecast_aligned = sarima_forecast_mean[:len(actual_values)].values

# Calculate error metrics for SARIMA
sarima_mae, sarima_mse, sarima_rmse = calculate_metrics(actual_values[:len(sarima_forecast_aligned)], sarima_forecast_aligned)

# Align forecast with actual data length for FB Prophet
prophet_forecast_aligned = prophet_forecast_df['yhat'][:len(actual_values)].values

# Calculate error metrics for FB Prophet
prophet_mae, prophet_mse, prophet_rmse = calculate_metrics(actual_values[:len(prophet_forecast_aligned)], prophet_forecast_aligned)

# Align forecast with actual data length for NeuralProphet
neuralprophet_forecast_aligned = neuralprophet_forecast_df['yhat1'][:len(actual_values)].values

# Calculate error metrics for NeuralProphet
neuralprophet_mae, neuralprophet_mse, neuralprophet_rmse = calculate_metrics(actual_values[:len(neuralprophet_forecast_aligned)], neuralprophet_forecast_aligned)



Duplicate dates found:
Empty DataFrame
Columns: [ds, y]
Index: []


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjdw3rv7t/iati2yqi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjdw3rv7t/dbr4xr6n.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=43679', 'data', 'file=/tmp/tmpjdw3rv7t/iati2yqi.json', 'init=/tmp/tmpjdw3rv7t/dbr4xr6n.json', 'output', 'file=/tmp/tmpjdw3rv7t/prophet_modelvjm82prl/prophet_model-20240518170356.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:03:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:03:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
WARNING - (NP.forecaster.fit) - When Glo

Finding best initial lr:   0%|          | 0/225 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency W-SAT corresponds to 99.899% of the data.
INFO:NP.df_utils:Major frequency W-SAT corresponds to 99.899% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO:NP.df_utils:Defined frequency is equal to major frequency - W
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.636% of the data.
INFO:NP.df_utils:Major frequency W-SUN corresponds to 99.636% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - W
INFO:NP.df_utils:Defined frequency is equal to major frequency - W
INFO - (NP.df_utils._infer_frequency) - Major frequency W-SUN corresponds to 99.636% of the data.
INFO:NP.df_utils:Major frequency W-SUN corresponds to 99.636% of the data.
INFO - (NP.df_utils._infer_frequency

Predicting: 31it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [12]:
# Plot comparison using plotly.graph_objs
fig = go.Figure()

# Add actual data
fig.add_trace(go.Scatter(x=df_california['ds'], y=df_california['y'], mode='markers', name='Actual Data'))

# Add SARIMA forecast
fig.add_trace(go.Scatter(x=sarima_forecast_df['ds'], y=sarima_forecast_df['yhat'], mode='lines', name='SARIMA Forecast'))

# Add FB Prophet forecast
fig.add_trace(go.Scatter(x=prophet_forecast_df['ds'], y=prophet_forecast_df['yhat'], mode='lines', name='Prophet Forecast'))

# Add NeuralProphet forecast
fig.add_trace(go.Scatter(x=neuralprophet_forecast_df['ds'], y=neuralprophet_forecast_df['yhat1'], mode='lines', name='NeuralProphet Forecast'))

# Update layout
fig.update_layout(
    title="Forecast Comparison",
    title_x=0.5,
    xaxis_title='Date',
    yaxis_title='Total ILI',
    xaxis=dict(range=[df_california['ds'].min(), end_date])
)

# Show the plot
fig.show()

# Create a DataFrame to store error metrics
metrics_df = pd.DataFrame({
    'Model': ['SARIMA', 'FB Prophet', 'NeuralProphet'],
    'MAE': [sarima_mae, prophet_mae, neuralprophet_mae],
    'MSE': [sarima_mse, prophet_mse, neuralprophet_mse],
    'RMSE': [sarima_rmse, prophet_rmse, neuralprophet_rmse]
})

# Display the error metrics
print(metrics_df)

           Model          MAE           MSE         RMSE
0         SARIMA   586.436844  6.147518e+05   784.061104
1     FB Prophet   300.789610  2.141324e+05   462.744389
2  NeuralProphet  1553.444236  2.528373e+06  1590.085919


# **The Results**
- FB Prophet: Despite the close alignment of the trend lines in the plot, FB Prophet's significantly lower error metrics indicate it handles the dataset's complexities more effectively, capturing the trends and seasonality better than NeuralProphet.
- NeuralProphet: The higher error metrics in the recent run suggest possible issues with tuning or sensitivity to specific data points. The model's previous better performance indicates potential for improvement with further tuning.
- SARIMA: Provides a balanced performance with trend alignment but falls short of FB Prophet in terms of error metrics, highlighting its limitations in handling complex seasonality compared to Prophet models.

# **Conclusion**
While the plot shows a visually close forecast between FB Prophet and NeuralProphet, the error metrics reveal FB Prophet's superior performance. Continued tuning and diagnostics of NeuralProphet are recommended to achieve results consistent with its potential shown in previous runs. SARIMA remains a strong contender but is outperformed by FB Prophet in this instance.